# Query Model Repository for best node embeddings model

In [1]:
from hops import model
from hops.model import Metric
MODEL_NAME="NodeEmbeddings"
EVALUATION_METRIC="accuracy"

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
4,application_1607676376502_0005,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [2]:
best_model = model.get_best_model(MODEL_NAME, EVALUATION_METRIC, Metric.MAX)

In [3]:
best_model['experimentId']

'application_1607676376502_0004_1'

# Send Prediction Requests to the Served Model using Hopsworks REST API

In [6]:
import tensorflow as tf
from tensorflow import keras  

import pandas as pd
from stellargraph import StellarDiGraph
from stellargraph.mapper import Node2VecLinkGenerator, Node2VecNodeGenerator
from stellargraph.data import UnsupervisedSampler, BiasedRandomWalk
from stellargraph.layer import Node2Vec
import pydoop.hdfs as pydoop
import hsfs
from hops import hdfs

In [7]:
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [8]:
node_td = fs.get_training_dataset("node_td", 1)
edge_td = fs.get_training_dataset("edges_td", 1)

In [9]:
# Get fg as pandas
node_pdf = node_td.read().toPandas()
edge_pdf = edge_td.read().drop("tran_timestamp").toPandas()

In [10]:
node_data = pd.DataFrame(node_pdf[['tx_behavior_id','prior_sar','initial_deposit']], index=node_pdf['id'])

print('Defining StellarDiGraph')
G =StellarDiGraph(node_data,
                      edges=edge_pdf, 
                      edge_type_column="tx_type")


Defining StellarDiGraph

In [11]:
walk_number = 2
walk_length = 2
batch_size = 1
emb_size = 16
# Extracting node embeddings
walker = BiasedRandomWalk(
        G,
        n=walk_number,
        length=walk_length,
        p=0.5,  # defines probability, 1/p, of returning to source node
        q=2.0,  # defines probability, 1/q, for moving to a node away from the source node
    )
unsupervised_samples = UnsupervisedSampler(G, nodes=list(G.nodes()), walker=walker)
generator = Node2VecLinkGenerator(G, batch_size)

node2vec = Node2Vec(emb_size, generator=generator)
x_inp, x_out = node2vec.in_out_tensors()

x_inp_src = x_inp[0]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

In [18]:
latest = tf.train.latest_checkpoint("hdfs:///Projects/{}/Experiments/".format(hdfs.project_name()) + best_model['experimentId'] + "/walk_number=2&walk_length=2&emb_size=16")
embedding_model.load_weights(latest)

In [16]:
print("hdfs:///Projects/{}/Experiments/".format(hdfs.project_name()) + best_model['experimentId'])
hdfs:///Projects/amlsim/Experiments/application_1607676376502_0004_1/

hdfs:///Projects/amlsim/Experiments/application_1607676376502_0004_1

In [13]:
nodes = list(G.nodes())
node_gen = Node2VecNodeGenerator(G, batch_size).flow(nodes)
node_embeddings = embedding_model.predict(node_gen)

In [14]:
node_embeddings[0].tolist()

[-0.47788143157958984, 0.21858453750610352, -0.8615648746490479, -0.41790318489074707, -0.44326281547546387, -0.04246211051940918, 0.01951909065246582, -0.4893348217010498, 0.989424467086792, 0.9611763954162598, 0.9251675605773926, -0.953972339630127, 0.81076979637146, -0.5028789043426514, -0.6400127410888672, -0.18201541900634766]

In [15]:
import numpy as np
from pyspark.mllib.linalg import Vectors, VectorUDT
from pyspark.mllib.regression import LabeledPoint

df = np.concatenate([np.random.randint(0,2, size=(1000)), np.random.randn(1000), 3*np.random.randn(1000)+2, 6*np.random.randn(1000)-2]).reshape(1000,-1)
df = map(lambda x: LabeledPoint(x[0], Vectors.dense(x[1:])), df)

mydf = spark.createDataFrame(df,["label", "features"])